In [55]:
from scipy.stats import chi2_contingency, power_divergence
import numpy as np

In [25]:
table = np.array([[338, 363],
         [125, 156]])

In [18]:
chi2_contingency(table, lambda_="log-likelihood")

(0.9783013059786239,
 0.3226185464764574,
 1,
 array([[330.51221996, 370.48778004],
        [132.48778004, 148.51221996]]))

In [41]:
chi2_contingency(table, correction=False, lambda_="log-likelihood")

(1.1234566202986294,
 0.28917536983502934,
 1,
 array([[330.51221996, 370.48778004],
        [132.48778004, 148.51221996]]))

In [48]:
_, _, _ ,e = chi2_contingency(table)

In [45]:
e = e.flatten()

In [37]:
np.array([(np.abs(o - ei) - 0.5) ** 2 / ei for o, ei in zip(table.flatten(), e.flatten())]).sum()

0.9768777737205301

In [57]:
from scipy.stats import chi2

In [68]:
chi2(1).cdf(1)

0.6826894921370859

In [49]:
e = e.flatten()
if (e[0]*e[3] - e[1]*e[2]) > 0:
    e[0] -= 0.5
    e[3] -= 0.5
    e[1] += 0.5
    e[3] += 0.5
else:
    e[0] += 0.5
    e[3] += 0.5
    e[1] -= 0.5
    e[3] -= 0.5

In [50]:
2 * np.array([o * np.log(o/ei) for o, ei in zip(table.flatten(), e.flatten())]).sum()

1.1677571703405292

In [51]:
from functools import reduce

In [52]:
def margins(a):
    margsums = []
    ranged = list(range(a.ndim))
    for k in ranged:
        marg = np.apply_over_axes(np.sum, a, [j for j in ranged if j != k])
        margsums.append(marg)
    return margsums

In [53]:
def expected_freq(observed):
    # Typically `observed` is an integer array. If `observed` has a large
    # number of dimensions or holds large values, some of the following
    # computations may overflow, so we first switch to floating point.
    observed = np.asarray(observed, dtype=np.float64)

    # Create a list of the marginal sums.
    margsums = margins(observed)

    # Create the array of expected frequencies.  The shapes of the
    # marginal sums returned by apply_over_axes() are just what we
    # need for broadcasting in the following product.
    d = observed.ndim
    expected = reduce(np.multiply, margsums) / observed.sum() ** (d - 1)
    return expected

In [54]:
def chi2_contingency(observed, correction=True, lambda_=None):
    observed = np.asarray(observed)

    expected = expected_freq(observed)
    if np.any(expected == 0):
        # Include one of the positions where expected is zero in
        # the exception message.
        zeropos = list(zip(*np.nonzero(expected == 0)))[0]
        raise ValueError("The internally computed table of expected "
                         "frequencies has a zero element at %s." % (zeropos,))

    # The degrees of freedom
    dof = expected.size - sum(expected.shape) + expected.ndim - 1

    if dof == 0:
        # Degenerate case; this occurs when `observed` is 1D (or, more
        # generally, when it has only one nontrivial dimension).  In this
        # case, we also have observed == expected, so chi2 is 0.
        chi2 = 0.0
        p = 1.0
    else:
        if dof == 1 and correction:
            # Adjust `observed` according to Yates' correction for continuity.
            # Magnitude of correction no bigger than difference; see gh-13875
            diff = expected - observed
            direction = np.sign(diff)
            magnitude = np.minimum(0.5, np.abs(diff))
            observed = observed + magnitude * direction

        chi2, p = power_divergence(observed, expected,
                                   ddof=observed.size - 1 - dof, axis=None,
                                   lambda_=lambda_)

    return chi2, p, dof, expected